## Ant Colony Optimization algorithm  (ACOA) as an optimizer for training a NN

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd
import math
import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt

# importing library for support vector machine classifier
from sklearn.svm import SVC
# importing library for K-neares-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
# importing library for Linear Discriminant Analysis Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# importing library for Quadratic Discriminant Analysis Model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from keras.layers import Dense # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained model

from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model

# representation of model layers
from keras.utils.vis_utils import plot_model
from numpy.random import choice as np_choice
import copy

In [ ]:
path="datasets/bin_data.csv"
bin_data=pd.read_csv(path)
bin_data

In [ ]:
X = bin_data.iloc[:,0:93].values # dataset excluding target attribute (encoded, one-hot-encoded,original)
Y = bin_data[['intrusion']].values # target attribute

In [ ]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [ ]:
def Fobj():
    mlp = Sequential() # creating model
    # adding input layer and first layer with 50 neurons
    # X_train.shape is (94479, 93)
    mlp.add(Dense(units=33, input_dim=X_train.shape[1], activation='relu'))#50x93=4650
    # output layer with sigmoid activation
    mlp.add(Dense(units=1,activation='sigmoid'))
    return mlp
# len(Fobj().get_weights()[1])
# Fobj().get_weights()[0].flatten().shape[0]

In [ ]:
mlp=Fobj()
# defining loss function, optimizer, metrics and then compiling model
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
dim=0;
for i in range(len(mlp.get_weights())):# 4 for 4 array 
    dim=dim+mlp.get_weights()[i].flatten().shape[0]# 4650+50+50+1=4751
# dim = 4751
# print(math.sqrt(dim))
N=20# Population size
X=np.zeros((N,dim)) #(20, 4751)
# print(X[:,:3])
Fit=np.ones(N)*float('inf') #(20, 1)
# print(Fit.reshape(N,1))
bF=float('inf')
bPos=[];
# print(len(mlp.get_weights()))#4
# len(mlp.layers)#2

In [ ]:
# get wights and bias Y=XW+bias 
def setWeights(X):
    Len=0
    # X is each row data in datasets X and len is 4751
    # len(mlp.layers is 2 each contain 4 array
    for i in range(0,len(mlp.layers)):
#         get Wights first from layers
        a=mlp.layers[i].get_weights()[0].shape
        weights=X[Len:(Len+mlp.layers[i].get_weights()[0].flatten().shape[0])]
        weights=np.reshape(weights,a) 
        Len=Len+mlp.layers[i].get_weights()[0].flatten().shape[0]
#         get bias first from layers
        a=mlp.layers[i].get_weights()[1].shape
        bias=X[Len:(Len+mlp.layers[i].get_weights()[1].flatten().shape[0])]
        bias=np.reshape(bias,a)
        Len=Len+mlp.layers[i].get_weights()[1].flatten().shape[0]
        mlp.layers[i].set_weights([weights,bias])
#         print(len(mlp.get_weights()))#4
#         len(mlp.layers)#4


In [ ]:
def check_boundaries(X):
    for j in range(dim):
        if X[j]>U_bounds:
            X[j]=(L_bounds+np.random.random(1)*(U_bounds-L_bounds))
        elif  X[j]<L_bounds:
            X[j]=(L_bounds+np.random.random(1)*(U_bounds-L_bounds))

In [ ]:
L_bounds=0;
U_bounds=1;
def initialization():
    for i in range(N):
        for j in range(dim):
            X[i][j]=(L_bounds+np.random.rand(1)*(U_bounds-L_bounds))
        setWeights(X[i,:])
        print("------------Solution ",i,"-------------")
        score=mlp.evaluate(X_train, y_train, verbose = 1) 
        Fit[i]= score[0]#loss
        print(score[1])#accurcy


In [ ]:
#get numner node visted and probility
def pick_move(distances,pheromone, visited,alpha=1,beta=1):
    all_inds = range(len(distances))
    pheromone = np.copy(pheromone)
    # print(pheromone)
    pheromone[list(visited)] = 0
    # print(pheromone[list(visited)])
    row = pheromone ** alpha * (( 1.0 / distances) ** beta)
    # h=f"""
    # {pheromone} ** {alpha} * (( 1.0 / {dist}) ** {beta})
    # """
    # print(h)
    # print(row)
    norm_row = row / row.sum()#probility
    # print(norm_row)
    move = np_choice(all_inds, 1, p=norm_row)[0]
    return move #3

In [ ]:
#get path each node
def gen_path(distances,pheromone,start=0):
    path = []
    visited = set()
    visited.add(start)
    prev = start
    for i in range(len(distances) - 1):
        move = pick_move(distances[prev],pheromone[prev], visited)
        path.append((prev, move))
        prev = move
        visited.add(move)
        # print(f"move: {move} prev:{prev} visited: {visited}")
        # print(f"-- path {path}")
    path.append((prev, start)) # going back to where we started   
    # print(f"path {path}")
    return path

In [ ]:
#get distance each node
def gen_path_dist(distances,path):
    total_dist = 0
    # get values shortest path after
    total_value=list()
    for ele in path:
          total_value.append(distances[ele])
          total_dist +=distances[ele]
    #     print(f"{ele} : {distances[ele]}")
    # print(f"total_dist: {total_dist}")
    return total_dist,total_value

In [ ]:
#get distance each node and path in all_path
def gen_all_paths(distances,pheromone,n_ants):
    all_paths = []
    for i in range(n_ants):
        path = gen_path(distances,pheromone)
        genPathDist,_=gen_path_dist(distances,path)
        all_paths.append((path, genPathDist))
    return all_paths

In [ ]:
def spread_pheronome(distances,pheromone ,all_paths, n_best, shortest_path):
    # sorted the paths by distance
    sorted_paths = sorted(all_paths, key=lambda x: x[1])
    # print(f"sorted_paths : {sorted_paths}")
    for path, dist in sorted_paths[:n_best]:
        for move in path:
            #update pheromone values 
            pheromone[move] += 1.0 / distances[move]
    return pheromone

In [ ]:
def run(distances, n_ants, n_best, n_iterations, decay):
    all_inds = range(len(distances))
    pheromone = np.ones(distances.shape) / len(distances)
    shortest_path = None
    all_time_shortest_path = ("placeholder", np.inf)
    for i in range(n_iterations):
        #get distance each node and path in all_path
        all_paths = gen_all_paths(distances,pheromone,n_ants)
        # print(f"all_paths : {all_paths}")
        pheromone=spread_pheronome(distances,pheromone ,all_paths, n_best, shortest_path=shortest_path)
        shortest_path = min(all_paths, key=lambda x: x[1])
#         print (shortest_path)
        # swap the smallest value of shortest_path
        if shortest_path[1] < all_time_shortest_path[1]:
            all_time_shortest_path = shortest_path            
        pheromone = pheromone * decay 
    return all_time_shortest_path